In [ ]:
"""
PART ONE OF THREE v2

COMPANY : HP

1. Get all product links from company product list

"""

In [65]:
import os
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from time import sleep
from bs4 import BeautifulSoup


In [50]:
def set_directory(directory):
    os.chdir(directory)

In [51]:
def initialize_webdriver():
    options = Options()
    #options.add_argument('--headless')  # Optionally run in headless mode
    options.add_argument('--no-sandbox')  # Bypass OS security model
    options.add_argument('--disable-gpu')  # GPU hardware acceleration isn't necessary
    options.add_argument('start-maximized')  # Start maximized to avoid any hidden elements issues
    options.add_argument('disable-infobars')
    options.add_argument('--disable-extensions')
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
    return driver

In [52]:
def load_page(driver, url):
    driver.get(url)
    sleep(5)
    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.TAG_NAME, 'body')))

In [53]:
def dismiss_region_selection_popup(driver):
    try:
        no_thanks_button = WebDriverWait(driver, 5).until(
            EC.element_to_be_clickable((By.CSS_SELECTOR, 'button.optly-country-continue[data-optly-event="e_continue_country"]'))
        )
        no_thanks_button.click()
    except Exception as e:
        print("No 'NO THANKS' button found or error: ", str(e))

In [54]:
def scroll_page(driver):
    """Scroll the page down by 500 pixels."""
    driver.execute_script("window.scrollBy(0, 250);")
    sleep(2)  # Wait for the page to load more items

In [55]:
def scroll_page_up(driver):
    """Scroll the page up by 100 pixels."""
    driver.execute_script("window.scrollBy(0, -500);")
    #sleep(.5)  # Wait for the page to adjust

In [62]:
def collect_links(driver, existing_urls):
    """Collect product links from the page and return them."""
    links = driver.find_elements(By.CSS_SELECTOR, 'a.ProductTitle-module_link__3V6Pz')
    new_links = []
    for link in links:
        href = link.get_attribute('href')
        if "shop/pdp" in href and href not in existing_urls and not href.endswith("#reviews"):
            print(href)
            existing_urls.add(href)
            new_links.append(href)
    return new_links


In [63]:
def handle_load_more(driver):
    """Handle clicking the 'Load More' button if it is present."""
    try:
        load_more_button = WebDriverWait(driver, 5).until(
            EC.visibility_of_element_located((By.CSS_SELECTOR, 'button[data-gtm-value="load-more"]'))
        )
        driver.execute_script("arguments[0].scrollIntoView();", load_more_button)
        load_more_button.click()
        print('---------------------MORE PRODUCTS LOADED---------------------')
        return True
    except:
        print("No more 'Load More' button found or error.")
        return False

In [68]:
def main():
    # Configuration
    info_location = r'C:\Users\keena\Desktop\Comp Sys Project'
    url = "https://www.hp.com/us-en/shop/vwa/business-solutions/brand=Dragonfly,Elite,EliteBook,EliteDesk,EliteOne,Folio,Pro,ProBook,ProDesk,ProOne,Z,Z-by-HP,ZBook?jumpid=ma_home_bar_na_5_240609"
    company_name = 'HP'
    save_file_name = company_name + '_product_links.txt'

    # Set the working directory
    set_directory(info_location)

    # Initialize the WebDriver
    driver = initialize_webdriver()
    
    existing_urls = set()

    try:
        # Load the page and handle the popup
        load_page(driver, url)
        dismiss_region_selection_popup(driver)

        # Scroll the page and collect links
        while True:
            scroll_page(driver)
            sleep(1)
            if not handle_load_more(driver):
                break  # Exit the loop if no "Load More" button is found or there's an error

        # Scroll back up to double-check for any missed links
        while True:
            scroll_page_up(driver)
            sleep(.2)
            if driver.execute_script("return window.pageYOffset;") == 0:
                break  # Stop scrolling up when we reach the top of the page
        
        # Extract the final page source
        page_source = driver.page_source

        # Parse with BeautifulSoup
        soup = BeautifulSoup(page_source, 'html.parser')
        product_links = soup.select('a.ProductTitle-module_link__3V6Pz')

        # Collect and save links
        all_links = []
        for link in product_links:
            href = link.get('href')
            if "shop/pdp" in href and href not in existing_urls and not href.endswith("#reviews"):
                print(href)
                all_links.append(href)

        print(f'{len(all_links)} Product Links Collected')
        # Write all collected links to the file at once
        with open(save_file_name, 'w') as file:
            for href in all_links:
                file.write("https://www.hp.com" + href + '\n')
            
    except Exception as e:
        print("Error during page interaction: ", str(e))
    finally:
        # Close the driver
        driver.quit()

if __name__ == "__main__":
    main()


---------------------MORE PRODUCTS LOADED---------------------
---------------------MORE PRODUCTS LOADED---------------------
---------------------MORE PRODUCTS LOADED---------------------
---------------------MORE PRODUCTS LOADED---------------------
---------------------MORE PRODUCTS LOADED---------------------
---------------------MORE PRODUCTS LOADED---------------------
---------------------MORE PRODUCTS LOADED---------------------
---------------------MORE PRODUCTS LOADED---------------------
---------------------MORE PRODUCTS LOADED---------------------
---------------------MORE PRODUCTS LOADED---------------------
---------------------MORE PRODUCTS LOADED---------------------
No more 'Load More' button found or error.
/us-en/shop/pdp/hp-z2-tower-g9-workstation-wolf-pro-security-edition-p-a1qh3ua-aba-1
/us-en/shop/pdp/hp-elite-mini-800-g9-desktop-pc-p-88u24ua-aba-1
/us-en/shop/pdp/hp-elitebook-860-16-inch-g10-notebook-pc-wolf-pro-security-edition
/us-en/shop/pdp/hp-zbook-firefly